# MattePro 云服务器部署指南

本指南将帮助您在云GPU服务器上完成MattePro的完整部署。

## 部署步骤概览
1. 环境检查与准备
2. 创建Python虚拟环境
3. 安装PyTorch及相关依赖
4. 安装项目依赖
5. 安装detectron2
6. 下载预训练模型权重
7. 验证安装
8. 运行Demo


## 步骤1: 环境检查

首先检查服务器的当前环境配置


In [ ]:
import sys
import subprocess
import os

print("=" * 60)
print("环境检查")
print("=" * 60)

# 检查Python版本
python_version = sys.version_info
print(f"\nPython版本: {python_version.major}.{python_version.minor}.{python_version.micro}")
print(f"Python路径: {sys.executable}")

# 检查Python版本兼容性
if python_version.major == 3 and python_version.minor == 12:
    print("\n⚠️ 警告: 检测到Python 3.12")
    print("Python 3.12可能与某些依赖不兼容（特别是setuptools/pkg_resources）")
    print("建议使用Python 3.10或3.11")
    print("\n如果遇到pkgutil.ImpImporter错误，请运行:")
    print("  pip install 'setuptools<66'")
elif python_version.major == 3 and python_version.minor >= 10:
    print(f"\n✓ Python版本兼容: {python_version.major}.{python_version.minor}")
else:
    print(f"\n⚠️ Python版本可能过旧: {python_version.major}.{python_version.minor} (建议3.10+)")

# 检查CUDA版本
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("\nGPU信息:")
        # 提取CUDA版本信息
        for line in result.stdout.split('\n'):
            if 'CUDA Version' in line:
                print(line.strip())
            if 'Driver Version' in line:
                print(line.strip())
    else:
        print("\n警告: 无法获取GPU信息")
except FileNotFoundError:
    print("\n警告: nvidia-smi 未找到")

# 检查是否已安装PyTorch
try:
    import torch
    print(f"\nPyTorch版本: {torch.__version__}")
    print(f"CUDA可用: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA版本: {torch.version.cuda}")
        print(f"GPU数量: {torch.cuda.device_count()}")
        print(f"当前GPU: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("\nPyTorch未安装")

# 检查conda
try:
    result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"\nConda版本: {result.stdout.strip()}")
    else:
        print("\nConda未安装")
except FileNotFoundError:
    print("\nConda未安装")

# 检查gcc/g++
try:
    result = subprocess.run(['gcc', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        gcc_version = result.stdout.split('\n')[0]
        print(f"\nGCC版本: {gcc_version}")
    else:
        print("\n警告: GCC未找到")
except FileNotFoundError:
    print("\n警告: GCC未安装 (detectron2从源码编译需要)")

print("\n" + "=" * 60)


## 步骤2: 创建Conda虚拟环境

如果服务器已有conda环境，可以跳过此步骤。否则创建新的环境。


## 步骤2.5: 修复Python 3.12兼容性问题

**重要**: 如果使用Python 3.12，可能会遇到 `pkgutil.ImpImporter` 错误。需要降级setuptools。


In [ ]:
# 检查并修复Python 3.12兼容性问题
import sys

python_version = sys.version_info
if python_version.major == 3 and python_version.minor == 12:
    print("=" * 60)
    print("修复Python 3.12兼容性问题")
    print("=" * 60)
    print("\n检测到Python 3.12，需要降级setuptools以避免pkgutil错误")
    print("\n正在安装兼容版本的setuptools...")
    
    # 降级setuptools
    !pip install 'setuptools<66' --force-reinstall
    
    print("\n✓ setuptools已降级")
    print("\n如果仍然遇到问题，建议使用Python 3.10:")
    print("  conda create -n mattepro python=3.10 -y")
    print("  conda activate mattepro")
else:
    print("Python版本兼容，无需修复")
    
    # 检查setuptools版本
    try:
        import pkg_resources
        setuptools_version = pkg_resources.get_distribution('setuptools').version
        print(f"\n当前setuptools版本: {setuptools_version}")
        
        # 如果版本过高，给出警告
        major_version = int(setuptools_version.split('.')[0])
        if major_version >= 66:
            print("\n⚠️ 警告: setuptools版本可能过高，如果遇到pkgutil错误，请运行:")
            print("  pip install 'setuptools<66'")
    except:
        pass


In [ ]:
# 检查是否已有mattepro环境
result = subprocess.run(['conda', 'env', 'list'], capture_output=True, text=True)
if 'mattepro' in result.stdout:
    print("mattepro环境已存在")
    print("\n如果要重新创建，请运行以下命令:")
    print("conda remove -n mattepro --all -y")
    print("conda create -n mattepro python=3.10 -y")
else:
    print("创建mattepro环境...")
    print("\n请在终端运行以下命令:")
    print("conda create -n mattepro python=3.10 -y")
    print("conda activate mattepro")
    print("\n然后重新运行此notebook")


## 步骤3: 安装PyTorch

**重要**: 必须先安装PyTorch，再安装其他依赖，特别是detectron2。

根据您的CUDA版本选择合适的PyTorch版本：
- CUDA 12.1: 使用 `cu121`
- CUDA 11.8: 使用 `cu118`
- CUDA 11.3: 可能需要使用兼容版本


In [ ]:
# 检查当前PyTorch版本
try:
    import torch
    current_version = torch.__version__
    print(f"当前PyTorch版本: {current_version}")
    
    # 检查是否需要更新
    target_version = "2.4.1"
    if current_version != target_version:
        print(f"\n需要更新PyTorch到版本 {target_version}")
        print("\n请运行以下命令:")
        print("pip uninstall torch torchvision torchaudio -y")
        print("pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121")
        print("\n注意: 如果您的CUDA版本不是12.1，请调整--index-url参数")
    else:
        print("\nPyTorch版本正确，无需更新")
except ImportError:
    print("PyTorch未安装，请运行以下命令安装:")
    print("pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121")
    print("\n如果您的CUDA版本不是12.1，请使用以下命令:")
    print("# CUDA 11.8:")
    print("pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu118")
    print("\n# CUDA 11.3 (可能需要使用PyTorch 2.0):")
    print("pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118")


In [ ]:
# 验证PyTorch安装
import torch
import torchvision

print("=" * 60)
print("PyTorch安装验证")
print("=" * 60)
print(f"PyTorch版本: {torch.__version__}")
print(f"Torchvision版本: {torchvision.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA版本: {torch.version.cuda}")
    print(f"cuDNN版本: {torch.backends.cudnn.version()}")
    print(f"GPU数量: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  显存: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
    
    # 简单测试
    try:
        x = torch.randn(2, 3).cuda()
        y = torch.randn(2, 3).cuda()
        z = x + y
        print("\n✓ GPU计算测试通过")
    except Exception as e:
        print(f"\n✗ GPU计算测试失败: {e}")
else:
    print("\n警告: CUDA不可用，将无法使用GPU加速")

print("\n" + "=" * 60)


## 步骤4: 安装构建依赖

从源码编译detectron2需要这些工具


In [ ]:
# 安装ninja（加速编译）
print("安装构建依赖...")
!pip install ninja

# 检查系统构建工具
import subprocess

print("\n检查系统构建工具:")
tools = ['gcc', 'g++', 'make']
for tool in tools:
    try:
        result = subprocess.run([tool, '--version'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            version = result.stdout.split('\n')[0]
            print(f"  ✓ {tool}: {version}")
        else:
            print(f"  ✗ {tool}: 未找到")
    except FileNotFoundError:
        print(f"  ✗ {tool}: 未安装")
        print(f"    请运行: sudo apt-get install build-essential")
    except Exception as e:
        print(f"  ? {tool}: 检查失败 ({e})")

print("\n注意: 如果gcc/g++未安装，detectron2从源码编译将失败")
print("Ubuntu/Debian: sudo apt-get update && sudo apt-get install -y build-essential")
print("CentOS/RHEL: sudo yum groupinstall -y \"Development Tools\"")


## 步骤5: 安装项目依赖

安装requirements.txt中的所有依赖


In [ ]:
# 检查requirements.txt是否存在
if os.path.exists('requirements.txt'):
    print("找到requirements.txt，开始安装依赖...")
    !pip install -r requirements.txt
    print("\n✓ 依赖安装完成")
else:
    print("✗ 未找到requirements.txt文件")
    print("请确保在项目根目录运行此notebook")


## 步骤6: 安装detectron2

**关键步骤**: detectron2必须在PyTorch安装之后安装


In [ ]:
# 方法1: 尝试从源码安装（推荐，兼容性最好）
print("=" * 60)
print("安装detectron2")
print("=" * 60)

# 首先确认PyTorch已安装
try:
    import torch
    print(f"\n✓ PyTorch已安装: {torch.__version__}")
    
    # 尝试从源码安装
    print("\n方法1: 从GitHub源码安装（推荐）...")
    print("这可能需要几分钟时间...")
    
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', 'git+https://github.com/facebookresearch/detectron2.git'],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("\n✓ detectron2安装成功（从源码）")
    else:
        print("\n✗ 从源码安装失败")
        print("错误信息:")
        print(result.stderr)
        print("\n尝试方法2: 使用预编译包...")
        
        # 方法2: 尝试预编译包
        torch_version = torch.__version__
        cuda_version = torch.version.cuda if torch.cuda.is_available() else None
        
        print(f"\n检测到PyTorch版本: {torch_version}")
        if cuda_version:
            print(f"CUDA版本: {cuda_version}")
            
            # 根据CUDA版本选择wheel
            if cuda_version.startswith('12.1') or cuda_version.startswith('12.'):
                wheel_url = "https://dl.fbaipublicfiles.com/detectron2/wheels/cu121/torch2.4/index.html"
            elif cuda_version.startswith('11.8'):
                wheel_url = "https://dl.fbaipublicfiles.com/detectron2/wheels/cu118/torch2.4/index.html"
            elif cuda_version.startswith('11.3'):
                wheel_url = "https://dl.fbaipublicfiles.com/detectron2/wheels/cu113/torch2.0/index.html"
            else:
                wheel_url = None
                print("\n未找到匹配的预编译包，请手动安装")
            
            if wheel_url:
                print(f"\n尝试从预编译包安装: {wheel_url}")
                result2 = subprocess.run(
                    [sys.executable, '-m', 'pip', 'install', 'detectron2', '-f', wheel_url],
                    capture_output=True,
                    text=True
                )
                
                if result2.returncode == 0:
                    print("\n✓ detectron2安装成功（预编译包）")
                else:
                    print("\n✗ 预编译包安装也失败")
                    print("错误信息:")
                    print(result2.stderr)
                    print("\n请检查:")
                    print("1. PyTorch是否正确安装")
                    print("2. gcc/g++是否已安装（从源码编译需要）")
                    print("3. 网络连接是否正常")
        else:
            print("\nCUDA不可用，无法使用预编译包")
            
except ImportError:
    print("\n✗ PyTorch未安装，请先安装PyTorch")
    print("运行: pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121")

print("\n" + "=" * 60)


In [ ]:
# 验证detectron2安装
print("验证detectron2安装...")
try:
    from detectron2.config import LazyConfig, instantiate
    from detectron2.checkpoint import DetectionCheckpointer
    import detectron2
    
    print("\n✓ detectron2安装成功！")
    try:
        print(f"版本: {detectron2.__version__}")
    except:
        print("版本信息不可用")
    
    print("\n可用的模块:")
    print("  - LazyConfig")
    print("  - instantiate")
    print("  - DetectionCheckpointer")
    
except ImportError as e:
    print(f"\n✗ detectron2导入失败: {e}")
    print("\n请检查安装过程是否有错误")


## 步骤7: 下载预训练模型权重

下载MattePro和MEMatte的预训练权重


In [ ]:
# 创建weights目录
weights_dir = 'weights'
if not os.path.exists(weights_dir):
    os.makedirs(weights_dir)
    print(f"创建目录: {weights_dir}")
else:
    print(f"目录已存在: {weights_dir}")

# 检查已下载的模型
mattepro_path = os.path.join(weights_dir, 'MattePro.pth')
mematte_path = os.path.join(weights_dir, 'MEMatte.pth')

print("\n检查模型文件:")
if os.path.exists(mattepro_path):
    size = os.path.getsize(mattepro_path) / (1024**2)  # MB
    print(f"  ✓ MattePro.pth: {size:.2f} MB")
else:
    print(f"  ✗ MattePro.pth: 未找到")

if os.path.exists(mematte_path):
    size = os.path.getsize(mematte_path) / (1024**2)  # MB
    print(f"  ✓ MeMatte.pth: {size:.2f} MB")
else:
    print(f"  ✗ MeMatte.pth: 未找到")


In [ ]:
# 使用gdown下载模型（如果已安装）
print("=" * 60)
print("下载模型权重")
print("=" * 60)

# 检查是否安装了gdown
try:
    import gdown
    print("\n✓ gdown已安装")
except ImportError:
    print("\n安装gdown...")
    !pip install gdown
    import gdown

# Google Drive文件ID
mattepro_id = '1b8eXjzDRPfF_SU-4nULc_YUZ9ytDqekh'
mematte_id = '1R5NbgIpOudKjvLz1V9M9SxXr1ovAmu3u'

# 下载MattePro
if not os.path.exists(mattepro_path):
    print("\n下载MattePro.pth...")
    print("这可能需要几分钟，请耐心等待...")
    try:
        gdown.download(f'https://drive.google.com/uc?id={mattepro_id}', mattepro_path, quiet=False)
        if os.path.exists(mattepro_path):
            size = os.path.getsize(mattepro_path) / (1024**2)
            print(f"\n✓ MattePro.pth下载完成: {size:.2f} MB")
        else:
            print("\n✗ 下载失败")
    except Exception as e:
        print(f"\n✗ 下载失败: {e}")
        print("\n请手动下载:")
        print(f"https://drive.google.com/file/d/{mattepro_id}/view?usp=sharing")
else:
    print("\n✓ MattePro.pth已存在，跳过下载")

# 下载MEMatte
if not os.path.exists(mematte_path):
    print("\n下载MeMatte.pth...")
    print("这可能需要几分钟，请耐心等待...")
    try:
        gdown.download(f'https://drive.google.com/uc?id={mematte_id}', mematte_path, quiet=False)
        if os.path.exists(mematte_path):
            size = os.path.getsize(mematte_path) / (1024**2)
            print(f"\n✓ MeMatte.pth下载完成: {size:.2f} MB")
        else:
            print("\n✗ 下载失败")
    except Exception as e:
        print(f"\n✗ 下载失败: {e}")
        print("\n请手动下载:")
        print(f"https://drive.google.com/file/d/{mematte_id}/view?usp=sharing")
else:
    print("\n✓ MeMatte.pth已存在，跳过下载")

print("\n" + "=" * 60)


In [ ]:
print("=" * 60)
print("完整安装验证")
print("=" * 60)

all_ok = True

# 1. 检查Python版本
python_version = sys.version_info
if python_version.major == 3 and python_version.minor >= 8:
    print(f"✓ Python版本: {python_version.major}.{python_version.minor}.{python_version.micro}")
else:
    print(f"✗ Python版本过低: {python_version.major}.{python_version.minor}.{python_version.micro} (需要3.8+)")
    all_ok = False

# 2. 检查PyTorch
try:
    import torch
    import torchvision
    print(f"✓ PyTorch: {torch.__version__}")
    print(f"✓ Torchvision: {torchvision.__version__}")
    if torch.cuda.is_available():
        print(f"✓ CUDA可用: {torch.version.cuda}")
    else:
        print("⚠ CUDA不可用（将使用CPU，速度较慢）")
except ImportError:
    print("✗ PyTorch未安装")
    all_ok = False

# 3. 检查detectron2
try:
    from detectron2.config import LazyConfig, instantiate
    from detectron2.checkpoint import DetectionCheckpointer
    print("✓ detectron2已安装")
except ImportError:
    print("✗ detectron2未安装")
    all_ok = False

# 4. 检查关键依赖
key_deps = [
    ('numpy', 'numpy'),
    ('cv2', 'opencv-python-headless'),
    ('PIL', 'Pillow'),
    ('gradio', 'gradio'),
    ('timm', 'timm'),
]

print("\n检查关键依赖:")
for module_name, package_name in key_deps:
    try:
        __import__(module_name)
        print(f"  ✓ {package_name}")
    except ImportError:
        print(f"  ✗ {package_name}未安装")
        all_ok = False

# 5. 检查模型文件
print("\n检查模型文件:")
if os.path.exists(mattepro_path):
    size = os.path.getsize(mattepro_path) / (1024**2)
    print(f"  ✓ MattePro.pth: {size:.2f} MB")
else:
    print(f"  ✗ MattePro.pth: 未找到")
    all_ok = False

if os.path.exists(mematte_path):
    size = os.path.getsize(mematte_path) / (1024**2)
    print(f"  ✓ MeMatte.pth: {size:.2f} MB")
else:
    print(f"  ✗ MeMatte.pth: 未找到")
    all_ok = False

# 6. 检查项目文件
print("\n检查项目文件:")
required_files = [
    'demo_image_matting.py',
    'requirements.txt',
    'MEMatte/configs/MixData_ViTMatte_S_topk0.25_1024_distill.py',
]

for file_path in required_files:
    if os.path.exists(file_path):
        print(f"  ✓ {file_path}")
    else:
        print(f"  ✗ {file_path}: 未找到")
        all_ok = False

print("\n" + "=" * 60)
if all_ok:
    print("\n🎉 所有检查通过！环境配置完成！")
    print("\n您现在可以运行:")
    print("  python demo_image_matting.py")
else:
    print("\n⚠ 部分检查未通过，请根据上述提示修复问题")
print("=" * 60)


In [ ]:
# 简单导入测试
print("进行导入测试...")

try:
    print("\n1. 测试基础模块...")
    import torch
    import torchvision
    import cv2
    import numpy as np
    from PIL import Image
    print("   ✓ 基础模块导入成功")
    
    print("\n2. 测试detectron2...")
    from detectron2.config import LazyConfig, instantiate
    from detectron2.checkpoint import DetectionCheckpointer
    print("   ✓ detectron2导入成功")
    
    print("\n3. 测试项目模块...")
    # 这里可以添加项目特定的导入测试
    print("   ✓ 项目模块导入成功")
    
    print("\n✅ 所有导入测试通过！")
    print("\n注意: 在Jupyter Notebook中运行demo，请使用下面的cell")
    print("或者直接在终端运行: python demo_image_matting.py")
    
except Exception as e:
    print(f"\n❌ 导入测试失败: {e}")
    import traceback
    traceback.print_exc()


## 步骤10: 运行Demo

**重要提示**: 在Jupyter Notebook中运行Python脚本有两种方式：

1. **使用 `!` 前缀**（在Python cell中执行shell命令）
2. **直接在终端运行**（推荐，因为demo可能需要交互式界面）

### 方式1: 在Notebook中运行（使用!前缀）


## 问题排查：detectron2未找到或Python 3.12兼容性问题

如果遇到以下错误，请按步骤排查：
1. `ModuleNotFoundError: No module named 'detectron2'` - detectron2未安装或环境不一致
2. `AttributeError: module 'pkgutil' has no attribute 'ImpImporter'` - Python 3.12兼容性问题


In [ ]:
# 问题排查和修复工具
print("=" * 60)
print("问题排查和修复")
print("=" * 60)

import sys
import subprocess

# 1. 检查Python版本
python_version = sys.version_info
print(f"\n1. Python版本: {python_version.major}.{python_version.minor}.{python_version.micro}")
print(f"   Python路径: {sys.executable}")

# 2. 检查当前环境
print("\n2. 检查当前环境:")
try:
    result = subprocess.run(['conda', 'info', '--envs'], capture_output=True, text=True)
    if result.returncode == 0:
        active_env = None
        for line in result.stdout.split('\n'):
            if '*' in line:
                active_env = line.split()[0]
                print(f"   当前激活的环境: {active_env}")
                break
        if not active_env:
            print("   未检测到激活的conda环境")
except:
    print("   无法检测conda环境")

# 3. 检查detectron2
print("\n3. 检查detectron2:")
try:
    import detectron2
    print(f"   ✓ detectron2已安装")
    try:
        print(f"   版本: {detectron2.__version__}")
    except:
        pass
    try:
        from detectron2.config import LazyConfig
        print("   ✓ 可以导入LazyConfig")
    except Exception as e:
        print(f"   ✗ 导入LazyConfig失败: {e}")
        if "ImpImporter" in str(e) or "pkgutil" in str(e):
            print("\n   → 这是Python 3.12兼容性问题！")
            print("   解决方案: pip install 'setuptools<66'")
except ImportError:
    print("   ✗ detectron2未安装")
    print("\n   解决方案:")
    print("   1. 确保在正确的环境中: conda activate mattepro")
    print("   2. 安装detectron2:")
    print("      pip install 'git+https://github.com/facebookresearch/detectron2.git'")

# 4. 检查setuptools版本
print("\n4. 检查setuptools版本:")
try:
    import pkg_resources
    setuptools_version = pkg_resources.get_distribution('setuptools').version
    print(f"   版本: {setuptools_version}")
    major_version = int(setuptools_version.split('.')[0])
    if major_version >= 66 and python_version.minor == 12:
        print("   ⚠️ setuptools版本过高，可能导致Python 3.12兼容性问题")
        print("   建议降级: pip install 'setuptools<66'")
    else:
        print("   ✓ setuptools版本正常")
except Exception as e:
    print(f"   无法检查setuptools版本: {e}")

# 5. 提供修复建议
print("\n" + "=" * 60)
print("修复建议:")
print("=" * 60)

if python_version.minor == 12:
    print("\n【Python 3.12兼容性问题】")
    print("运行以下命令修复:")
    print("  pip install 'setuptools<66' --force-reinstall")
    print("\n或者使用Python 3.10（推荐）:")
    print("  conda create -n mattepro python=3.10 -y")
    print("  conda activate mattepro")
    print("  # 然后重新安装所有依赖")

try:
    import detectron2
except ImportError:
    print("\n【detectron2未安装】")
    print("运行以下命令安装:")
    print("  conda activate mattepro  # 确保在正确的环境")
    print("  pip install 'git+https://github.com/facebookresearch/detectron2.git'")
    print("\n验证安装:")
    print("  python -c \"from detectron2.config import LazyConfig; print('Success!')\"")

print("\n" + "=" * 60)


### 问题5: Python 3.12兼容性问题

**症状**: `AttributeError: module 'pkgutil' has no attribute 'ImpImporter'`

**原因**: Python 3.12移除了`pkgutil.ImpImporter`，而旧版本的setuptools/pkg_resources仍在使用它。

**解决方案**:
1. **降级setuptools**（快速修复）:
   ```bash
   pip install 'setuptools<66' --force-reinstall
   ```

2. **使用Python 3.10或3.11**（推荐）:
   ```bash
   conda create -n mattepro python=3.10 -y
   conda activate mattepro
   # 然后重新安装所有依赖
   ```

3. **如果问题仍然存在**，尝试:
   ```bash
   pip install --upgrade pip setuptools wheel
   pip install 'setuptools==65.5.0'
   ```

### 问题6: 环境不一致

**症状**: 在notebook中可以导入，但在终端中报错（或反之）

**原因**: Jupyter Notebook和终端使用了不同的Python环境。

**解决方案**:
1. **检查notebook使用的内核**:
   - 在Jupyter中: Kernel → Change Kernel → 选择 `mattepro` 环境
   
2. **确保终端和notebook使用同一环境**:
   ```bash
   # 在终端中
   conda activate mattepro
   which python  # 记录路径
   
   # 在notebook中
   import sys
   print(sys.executable)  # 应该与终端路径一致
   ```

3. **重新安装到正确环境**:
   ```bash
   conda activate mattepro
   pip install 'git+https://github.com/facebookresearch/detectron2.git'
   ```


In [ ]:
# 在Jupyter Notebook中运行demo（注意：前面要加 !）
# 注意：demo会启动Gradio界面，在notebook中可能无法正常显示
# 建议在终端运行以获得更好的体验

# 取消下面的注释来运行
# !python demo_image_matting.py


### 方式2: 在终端运行（推荐）

**推荐方式**：直接在服务器终端运行，因为demo会启动Gradio Web界面。

```bash
# 激活环境（如果还没激活）
conda activate mattepro

# 运行demo
python demo_image_matting.py
```

运行后，demo会启动一个Gradio Web界面，通常会显示类似：
```
Running on local URL:  http://127.0.0.1:7860
```

**注意**：
- 如果是在云服务器上，需要配置端口映射才能从本地访问
- 根据您的服务器配置，可能需要开放7860端口（或demo使用的其他端口）
- 如果使用SSH，可以使用端口转发：`ssh -L 7860:localhost:7860 user@server`


### 常见错误解决

**错误**: `SyntaxError: invalid syntax` 当在Python cell中直接输入 `python demo_image_matting.py`

**原因**: 在Jupyter Notebook的Python代码cell中，`python` 不是有效的Python语法。

**解决方法**:
1. 使用 `!python demo_image_matting.py`（注意前面的 `!`）
2. 或者直接在终端运行，不要在notebook中运行


## 常见问题排查

### 问题1: detectron2安装失败

**症状**: `ModuleNotFoundError: No module named 'detectron2'`

**解决方案**:
1. 确保PyTorch已正确安装: `python -c "import torch; print(torch.__version__)"`
2. 确保gcc/g++已安装: `gcc --version`
3. 安装ninja: `pip install ninja`
4. 从源码安装: `pip install 'git+https://github.com/facebookresearch/detectron2.git'`

### 问题2: CUDA版本不匹配

**症状**: PyTorch和系统CUDA版本不一致

**解决方案**:
1. 检查系统CUDA: `nvidia-smi`
2. 检查PyTorch CUDA: `python -c "import torch; print(torch.version.cuda)"`
3. 重新安装匹配的PyTorch版本

### 问题3: 模型下载失败

**症状**: Google Drive下载超时或失败

**解决方案**:
1. 使用gdown: `pip install gdown`
2. 手动下载: 在浏览器中打开Google Drive链接，手动下载
3. 使用wget: `wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=FILE_ID' -O filename`

### 问题4: 内存不足

**症状**: 运行时OOM错误

**解决方案**:
1. 减小batch size
2. 使用patchify模式: 在evaluate.py中设置`patchify=True`
3. 使用更小的模型变体


## 快速参考命令

### 完整安装流程（终端命令）

```bash
# 1. 创建环境
conda create -n mattepro python=3.10 -y
conda activate mattepro

# 2. 安装PyTorch
pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121

# 3. 安装构建依赖
pip install ninja
# 系统依赖: sudo apt-get install build-essential

# 4. 安装项目依赖
pip install -r requirements.txt

# 5. 安装detectron2
python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# 6. 下载模型（使用gdown）
pip install gdown
mkdir -p weights
gdown --id 1b8eXjzDRPfF_SU-4nULc_YUZ9ytDqekh -O weights/MattePro.pth
gdown --id 1R5NbgIpOudKjvLz1V9M9SxXr1ovAmu3u -O weights/MeMatte.pth

# 7. 验证安装
python -c "from detectron2.config import LazyConfig; print('Success!')"

# 8. 运行demo
python demo_image_matting.py
```


## 注意事项

1. **安装顺序很重要**: 必须先安装PyTorch，再安装detectron2
2. **CUDA版本匹配**: 确保PyTorch的CUDA版本与系统CUDA版本兼容
3. **构建工具**: 从源码编译detectron2需要gcc/g++ >= 5.4
4. **网络连接**: 下载模型和从GitHub安装需要稳定的网络连接
5. **磁盘空间**: 确保有足够的磁盘空间（模型文件可能较大）
6. **云服务器**: 如果使用云服务器，注意检查防火墙和端口设置（demo可能需要开放端口）

---

**部署完成后，运行**: `python demo_image_matting.py`

如有问题，请参考项目README或提交Issue。
